In [7]:
from huggingface_hub import login
access_token = "secret-token"
login(token=access_token)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [8]:
import torch
import datasets
from peft import PeftModel
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, AutoModel, TFBertForQuestionAnswering,TFAutoModelWithLMHead

In [9]:
GPU_use = 0
st = "cuda:"+str(GPU_use)
torch.cuda.set_device(GPU_use)

In [10]:
ds = datasets.load_dataset('marcomaccarini/ds_robot_33_large')

In [12]:
trn = ds['train']

In [15]:
base_model = 'meta-llama/Meta-Llama-3-8B'
tokr = AutoTokenizer.from_pretrained(base_model)
model = AutoModelForCausalLM.from_pretrained("marcomaccarini/SynthLA", torch_dtype=torch.bfloat16, device_map=GPU_use,token=access_token)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:05<00:00,  1.29s/it]


In [16]:
fmt = """
Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
USER: {}
===
{}
ASSISTANT:"""

In [18]:
def sql_prompt(d): 
    return fmt.format(d["context"], d["question"])
def question(table, quest):
    tst = dict(**trn[8])
    tst['context'] = table
    tst['question'] = quest
    return sql_prompt(tst)

In [36]:
#t = 'table([ eof x: 421 y: -115 z: 125, gripper: open , black-cup x: 321 y: 217 z: 80, green-cube x: 425 y: -120 z: 80 or: 14 ])'
t = 'table([ eof x: 85 y: 179 z: 548, gripper: open , black-cup x: -54 y: -27 z: 80, white-cup x: -5 y: 59 z: 60, box x: -30 y: 34 z: 100, green-cylinder x: 25 y: -3 z: 80 or: 142, green-cube x: -390 y: -490 z: 80 or: 83, grey-cube x: 56 y: -22 z: 80 or: 96, red-cube x: -32 y: 58 z: 80 or: 157, yellow-ball x: -21 y: 30 z: 20 or: 41, banana x: 2 y: 53 z: 20 or: 9, remote x: -48 y: 31 z: 30 or: 69, pen x: -53 y: -59 z: 10 or: 174 ])'
q = 'pick green-cube and place to black-cup'

In [79]:
%time
test = question(t,q)
toks = tokr(test, return_tensors="pt")
res = model.generate(**toks.to(st), max_new_tokens=100, top_p = 0).to('cpu')

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


CPU times: user 17 μs, sys: 0 ns, total: 17 μs
Wall time: 32.4 μs


In [38]:
print(tokr.batch_decode(res)[0].replace("*","\n"))

<|begin_of_text|>
Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
USER: table([ eof x: 85 y: 179 z: 548, gripper: open, black-cup x: -54 y: -27 z: 80, white-cup x: -5 y: 59 z: 60, box x: -30 y: 34 z: 100, green-cylinder x: 25 y: -3 z: 80 or: 142, green-cube x: -390 y: -490 z: 80 or: 83, grey-cube x: 56 y: -22 z: 80 or: 96, red-cube x: -32 y: 58 z: 80 or: 157, yellow-ball x: -21 y: 30 z: 20 or: 41, banana x: 2 y: 53 z: 20 or: 9, remote x: -48 y: 31 z: 30 or: 69, pen x: -53 y: -59 z: 10 or: 174 ])
===
pick green-cube and place to black-cup
ASSISTANT: above:-390;-490;80+35;83;<|end_of_text|>
